<a href="https://colab.research.google.com/github/kpansett/vlr-project-quantization/blob/main/VLR_Project_Quantization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install onnx
# !pip install --upgrade onnxruntime==1.7.0
# !pip install --upgrade onnxruntime-tools
# !pip install --upgrade transformers
# !pip install timm

In [ ]:
# !pip uninstall onnxruntime
# !pip uninstall onnxruntime-gpu

In [ ]:
import logging
import numpy as np
import os
import random
import sys
import time
import torch

import onnx
import onnxruntime

import transformers

In [ ]:
# from transformers import BertTokenizer, BertModel

# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# model = BertModel.from_pretrained("bert-base-uncased")
# text = "Replace me by any text you'd like."

# encoded_input = tokenizer(text, return_tensors='pt')
# output = model(**encoded_input)

In [ ]:
# encoded_input["input_ids"]
# encoded_input["attention_mask"]

In [ ]:
# !gdown --id 1B3gzyzuDN1DU0lvt2kDz2nTTwSKWqzV5
# !gdown --id 1VolF9P9cPSuD8CZMjwbKW20wUrAIaEFK
# !gdown --id 1vhdtH3iFaoZuMqOGm-8YM-diPWVfRJzv
# !gdown --id 1bv6_pZOsXW53EhlwU0ZgSk03uzFI61pN
# !gdown --id 1XFz1Vtz7MCBLn4_1QEojhFJ5Iw3eH3X4

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# !git clone https://github.com/zengyan-97/X-VLM.git

In [ ]:
import os
import sys
import yaml
sys.path.append("X-VLM/")
from models.xvlm import *
from models.model_retrieval import *

In [ ]:
config_path = "/content/X-VLM/configs/Retrieval_coco.yaml"

with open(config_path, 'r') as stream:
    config = yaml.safe_load(stream)

In [ ]:
# !cd X-VLM && mkdir data && cd data && rm -rf bert-base-uncased

In [ ]:
# !sudo apt-get install git-lfs
# !cd X-VLM && mkdir data
# !cd X-VLM/data && git lfs install
# !cd X-VLM/data && git clone https://huggingface.co/bert-base-uncased

In [ ]:
config["text_config"] = os.path.join("X-VLM", config["text_config"])
config["vision_config"] = os.path.join("X-VLM", config["vision_config"])
config["text_encoder"] = os.path.join("X-VLM", config["text_encoder"])
config["train_file"][0] = os.path.join("X-VLM", config["train_file"][0])
config["val_file"] = os.path.join("X-VLM", config["val_file"])
config["test_file"] = os.path.join("X-VLM", config["test_file"])

In [ ]:
# config

In [ ]:
model = XVLM(config=config)

# model = XVLMBase(config=config, load_vision_params=False, load_text_params=False,
#                  use_contrastive_loss=True, use_matching_loss=True, use_mlm_loss=False, use_bbox_loss=True, 
#                  config_text=None)


/usr/local/lib/python3.7/dist-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [ ]:
# !cd X-VLM/data && wget https://github.com/SwinTransformer/storage/releases/download/v1.0.0/swin_base_patch4_window7_224_22k.pth

In [ ]:
# ckpt_rpath = "/content/4m_base_model_state_step_199999.th"
ckpt_rpath = "/content/checkpoint_best.pth"
model.load_pretrained(ckpt_rpath, config)

### Loading pretrained vision encoder
### Loading pretrained text encoder
load checkpoint from /content/checkpoint_best.pth
missing_keys:  []
unexpected_keys:  []


In [ ]:
# !pip install ruamel.yaml
# !pip install pycocoevalcap

In [ ]:
from dataset import *

In [ ]:
# model

In [ ]:
# config

In [ ]:
# quantize model
quantized_model = torch.quantization.quantize_dynamic(
    model, {torch.nn.Linear}, dtype=torch.qint8
)

#print(quantized_model)

def print_size_of_model(model):
    torch.save(model.state_dict(), "temp.p")
    print('Size (MB):', os.path.getsize("temp.p")/(1024*1024))
    os.remove('temp.p')

print_size_of_model(model)
print_size_of_model(quantized_model)

Size (MB): 829.4066343307495
Size (MB): 287.1118965148926


In [ ]:
# quantized_model

In [ ]:
import os
import sys
import tempfile
import torch
import torch.distributed as dist
import torch.nn as nn
import torch.optim as optim
import torch.multiprocessing as mp

from torch.nn.parallel import DistributedDataParallel as DDP

def setup(rank, world_size):
    os.environ['MASTER_ADDR'] = 'localhost'
    os.environ['MASTER_PORT'] = '12355'

    # initialize the process group
    dist.init_process_group("gloo", rank=rank, world_size=world_size)

def cleanup():
    dist.destroy_process_group()

# dist.init_process_group(backend="gloo", rank=1, world_size=1)
setup(rank=0, world_size=1)

In [ ]:
model.to(device)

XVLM(
  (vision_encoder): SwinTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4))
      (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0): BasicLayer(
        dim=128, input_resolution=(96, 96), depth=2
        (blocks): ModuleList(
          (0): SwinTransformerBlock(
            dim=128, input_resolution=(96, 96), num_heads=4, window_size=12, shift_size=0, mlp_ratio=4.0
            (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
            (attn): WindowAttention(
              dim=128, window_size=(12, 12), num_heads=4
              (qkv): Linear(in_features=128, out_features=384, bias=True)
              (attn_drop): Dropout(p=0.0, inplace=False)
              (proj): Linear(in_features=128, out_features=128, bias=True)
              (proj_drop): Dropout(p=0.0, inplace=False)
              (softmax): Softmax(dim=

In [ ]:
from models.tokenization_bert import BertTokenizer
from models.tokenization_roberta import RobertaTokenizer

In [ ]:
# !tar -xvf finetune.tar -C X-VLM/data/

In [ ]:
config["prompt"] = ""

In [ ]:
# !wget http://images.cocodataset.org/zips/val2014.zip
# !wget http://images.cocodataset.org/zips/train2014.zip

In [ ]:
# !mkdir images
# !cd images && mkdir coco
# !unzip val2014.zip -d images/coco/
# !unzip train2014.zip -d images/coco/

In [ ]:
# train_dataset, val_dataset, test_dataset = create_dataset('caption_coco', config)
train_dataset, val_dataset, test_dataset = create_dataset('re', config)

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:853: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "
/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:288: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "


In [ ]:
# train_dataset[0]

In [ ]:
samplers = [None, None, None]
train_loader, val_loader, test_loader = create_loader([train_dataset, val_dataset, test_dataset], samplers,
                                                      batch_size=[4,4,4],
                                                      num_workers=[4, 4, 4],
                                                      is_trains=[True, False, False],
                                                      collate_fns=[None, None, None])

In [ ]:
model = model.to(device)
print("### Total Params: ", sum(p.numel() for p in model.parameters() if p.requires_grad))

if config['use_roberta']:
    tokenizer = RobertaTokenizer.from_pretrained(config['text_encoder'])
else:
    tokenizer = BertTokenizer.from_pretrained(config['text_encoder'])


### Total Params:  213959547


In [ ]:
# texts = val_loader.dataset.text
# num_text = len(texts)
# text_bs = 2  # 256
# text_feats = []
# text_embeds = []  
# text_atts = []
# for i in range(0, num_text, text_bs):
#     text = texts[i: min(num_text, i + text_bs)]
#     text_input = tokenizer(text, padding='max_length', truncation=True, max_length=config['max_tokens'],
#                             return_tensors="pt").to(device)
#     text_output = model.text_encoder(text_input.input_ids, attention_mask=text_input.attention_mask, mode='text')
#     text_feat = text_output.last_hidden_state
#     text_embed = F.normalize(model.text_proj(text_feat[:, 0, :]))
#     text_embeds.append(text_embed)
#     text_feats.append(text_feat)
#     text_atts.append(text_input.attention_mask)
#     break

# text_embeds = torch.cat(text_embeds, dim=0)
# text_feats = torch.cat(text_feats, dim=0)
# text_atts = torch.cat(text_atts, dim=0)

In [ ]:
# text_atts.reshape(1, 2, 40)

In [ ]:
# encoded_input["input_ids"]
# encoded_input["attention_mask"]

# inputs = {'image': torch.rand(1,3, 384, 384, dtype=torch.float).cuda(),
#                     'text_ids': torch.ones(1, 2, 40, dtype=torch.int64).cuda(),
#                     'text_atts': text_atts.reshape(1, 2, 40)}

In [ ]:
# model(inputs['image'], encoded_input["input_ids"], encoded_input["attention_mask"])

In [ ]:
# for data in train_loader:
#     print(len(data))
#     break

# for data in val_loader:
#     print(data)
#     break

In [ ]:
train_data = np.load("train_loader.npz")
# train_data["images"], train_data["text_ids"], train_data["text_attn"]

In [ ]:
model.cuda()
start = time.time()
bs = 4
for idx in range(0, 500, bs):
    if idx >= 4:
      break
    inputs = {'image': torch.Tensor(train_data["images"][idx:idx+bs]),
          'text_ids': torch.LongTensor(train_data["text_ids"][idx:idx+bs]), 
          'text_atts': torch.Tensor(train_data["text_attn"][idx:idx+bs])}

    outputs = model(inputs["image"].cuda(), inputs["text_ids"].cuda(), inputs["text_atts"].cuda())

end = time.time()
print(end - start)

1.6367626190185547


In [ ]:
from models.tokenization_bert import BertTokenizer

for i, (image, text, id) in enumerate(train_loader):
    image = image.to(device, non_blocking=True)
    # print(text)
    # text = ["abc def"]*image.shape[0]
    # print(text)
    text_input = tokenizer(text, padding='longest', max_length=config['max_tokens'], return_tensors="pt").to(device)

    loss_itc, loss_itm = model(image, text_input.input_ids, text_input.attention_mask)
    break

In [ ]:
inputs = {'image': image.cuda(),
          'text_ids': text_input.input_ids, 
          'text_atts': text_input.attention_mask}

In [ ]:
def export_onnx_model(model, onnx_model_path):
    with torch.no_grad():
        # inputs = {'image': torch.rand(1,3, 384, 384, dtype=torch.float).cuda(),
        #             'text_ids': encoded_input["input_ids"].cuda(),
        #             'text_atts': encoded_input["attention_mask"].cuda()
        #           }
        outputs = model(**inputs)
        # outputs = model(inputs['text_ids'], inputs['text_atts'])                         # model input (or a tuple for multiple inputs)

        symbolic_names = {0: 'batch_size'}
        symbolic_names1 = {0: 'batch_size', 1: 'seq_len'}

        torch.onnx.export(model,                                            # model being run
                    (
                        inputs['image'],                             # model input (or a tuple for multiple inputs)
                    inputs['text_ids'],
                     inputs['text_atts']
                     ),                                         # model input (or a tuple for multiple inputs)
                    onnx_model_path,                                # where to save the model (can be a file or file-like object)
                    opset_version=13,                                 # the ONNX version to export the model to
                    do_constant_folding=True,                         # whether to execute constant folding for optimization
                    input_names=[
                                 'image',                         # the model's input names
                                'text_ids', 
                                'text_atts'
                                ],
                    output_names=['output'],                    # the model's output names
                    dynamic_axes={
                        'image': symbolic_names,        # variable length axes
                                'text_ids' : symbolic_names1,
                                'text_atts' : symbolic_names1,
                                'output' : symbolic_names
                                },
                          # operator_export_type=torch.onnx.OperatorExportTypes.ONNX_ATEN_FALLBACK
                          )
        # logger.info("ONNX Model exported to {0}".format(onnx_model_path))

# export_onnx_model(model, "xvlm2.onnx")

In [ ]:
def quantize_onnx_model(onnx_model_path, quantized_model_path):
    from onnxruntime.quantization import quantize_dynamic, QuantType
    import onnx
    onnx_opt_model = onnx.load(onnx_model_path)
    quantize_dynamic(onnx_model_path,
                     quantized_model_path,
                     weight_type=QuantType.QUInt8)

    # logger.info(f"quantized model saved to:{quantized_model_path}")

# quantize_onnx_model('xvlm2.onnx', 'xvlm2.opt.quant.onnx')

print('ONNX full precision model size (MB):', os.path.getsize("xvlm2.onnx")/(1024*1024))
print('ONNX quantized model size (MB):', os.path.getsize("xvlm2.opt.quant.onnx")/(1024*1024))

ONNX full precision model size (MB): 1079.7703914642334
ONNX quantized model size (MB): 300.550124168396


In [ ]:
# !pip uninstall onnxruntime-gpu
# !pip uninstall onnxruntime
# !pip install onnxruntime==1.7.0
# !pip install onnxruntime-gpu==1.7.0

In [ ]:
# from onnxruntime_tools import optimizer
# from onnxruntime_tools.transformers.onnx_model_bert import BertOptimizationOptions

# # disable embedding layer norm optimization for better model size reduction
# opt_options = BertOptimizationOptions('bert')
# opt_options.enable_embed_layer_norm = False

# opt_model = optimizer.optimize_model(
#     'bert.onnx',
#     'bert', 
#     num_heads=12,
#     hidden_size=768,
#     optimization_options=opt_options)
# opt_model.save_model_to_file('bert.opt.onnx')

In [ ]:
sess_options = onnxruntime.SessionOptions()
# sess_options.add_session_config_entry('session.load_model_format', 'ONNX')
sess_options.graph_optimization_level = onnxruntime.GraphOptimizationLevel.ORT_ENABLE_ALL
session = onnxruntime.InferenceSession('xvlm2.opt.quant.onnx', sess_options, providers=['CUDAExecutionProvider'])

In [ ]:
inputs = {'image': image.cpu().numpy(),
          'text_ids': text_input.input_ids.cpu().numpy(), 
          'text_atts': text_input.attention_mask.cpu().numpy()}

In [ ]:
# image.cpu().numpy()

In [ ]:
import time

In [ ]:
# images_all = []
# text_ids_all = []
# text_attn_all = []

# batch_count = 0
# for (image, text, id) in train_loader:
#     if batch_count >= 125:
#         break
#     images_all.append(image.numpy())
#     text_ids_all.append(text_input.input_ids.cpu().numpy())
#     text_attn_all.append(text_input.attention_mask.cpu().numpy())
#     batch_count += 1

# images_all = np.vstack(images_all)
# text_ids_all = np.vstack(text_ids_all)
# text_attn_all = np.vstack(text_attn_all)

In [ ]:
# np.savez("/content/drive/MyDrive/train_loader.npz", images=images_all, text_ids=text_ids_all, text_attn=text_attn_all)

In [ ]:
train_data = np.load("train_loader.npz")
# train_data["images"], train_data["text_ids"], train_data["text_attn"]

In [ ]:
# text_attn_all.shape

In [ ]:
for (image, text, id) in train_loader:
    inputs = {'image': image.numpy(),
          'text_ids': text_input.input_ids.cpu().numpy(), 
          'text_atts': text_input.attention_mask.cpu().numpy()}
    
    im = image.cuda()
    tid = text_input.input_ids
    ta = text_input.attention_mask
    
    break

In [ ]:
start = time.time()
bs = 4
for idx in range(0, 500, bs):
    if idx >=40:
      break
    inputs = {'image': train_data["images"][idx:idx+bs],
          'text_ids': train_data["text_ids"][idx:idx+bs], 
          'text_atts': train_data["text_attn"][idx:idx+bs]}

    # print(text_input.input_ids.shape)
    outputs = session.run(None, inputs)
    # break

end = time.time()
print(end - start)

55.480478286743164


In [ ]:
print(onnxruntime.get_device())

GPU


In [ ]:
model.cuda()
start = time.time()
bs = 10
for idx in range(0, 500, bs):
    inputs = {'image': torch.Tensor(train_data["images"][idx:idx+bs]),
          'text_ids': torch.LongTensor(train_data["text_ids"][idx:idx+bs]), 
          'text_atts': torch.Tensor(train_data["text_attn"][idx:idx+bs])}

    # print(text_input.input_ids.shape)
    outputs = model(inputs["image"].cuda(), inputs["text_ids"].cuda(), inputs["text_atts"].cuda())
    break
end = time.time()
print(end - start)

RuntimeError: ignored

In [ ]:
start = time.time()
for (image, text, id) in train_loader:
    inputs = {'image': image.cpu().numpy(),
          'text_ids': text_input.input_ids.cpu().numpy(), 
          'text_atts': text_input.attention_mask.cpu().numpy()}

    # print(text_input.input_ids.shape)
    outputs = session.run(None, inputs)
    break
end = time.time()
print(end - start)

5.425128221511841


In [ ]:
start = time.time()
count = 0
for (image, text, id) in train_loader:
    if count >= 125:
        break
    text_input = tokenizer(text, padding='longest', max_length=config['max_tokens'], return_tensors="pt").to(device)
    outputs = model(image.cuda(), text_input.input_ids, text_input.attention_mask)
    count += 1
end = time.time()
print(end - start)

20.372811555862427


In [ ]:
outputs

(tensor(9.4024, device='cuda:0', grad_fn=<DivBackward0>),
 tensor(5.4466, device='cuda:0', grad_fn=<NllLossBackward0>))

In [ ]:
model

In [ ]:
torch.__version__

'1.10.0+cu111'

* Quantization
* Data Loader 
    - Data: Flikr30k, Flikr8k
* Evaluation
* Presentation

In [ ]:
# !pip install libcudart==11.0.221
# !pip install libcufft==10.2.1.245
# !pip install libcurand==10.2.1.245
# !pip install libcublasLt==11.2.0.252
# !pip install libcublas==11.2.0.252
# !pip install libcudnn==8.0.4

In [ ]:
from Retrieval import *

In [ ]:
# model_without_ddp = quantized_model
model_without_ddp = model

In [ ]:
device = torch.device("cuda")

In [ ]:
model_without_ddp.to(device)

XVLM(
  (vision_encoder): SwinTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4))
      (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0): BasicLayer(
        dim=128, input_resolution=(96, 96), depth=2
        (blocks): ModuleList(
          (0): SwinTransformerBlock(
            dim=128, input_resolution=(96, 96), num_heads=4, window_size=12, shift_size=0, mlp_ratio=4.0
            (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
            (attn): WindowAttention(
              dim=128, window_size=(12, 12), num_heads=4
              (qkv): Linear(in_features=128, out_features=384, bias=True)
              (attn_drop): Dropout(p=0.0, inplace=False)
              (proj): Linear(in_features=128, out_features=128, bias=True)
              (proj_drop): Dropout(p=0.0, inplace=False)
              (softmax): Softmax(dim=

In [ ]:
# val_loader.dataset.text

In [ ]:
start_time = time.time()
print("Start evaluating", flush=True)

score_val_i2t, score_val_t2i, = evaluation(model_without_ddp, val_loader, tokenizer, device, config)
# score_test_i2t, score_test_t2i = evaluation(model_without_ddp, test_loader, tokenizer, device, config)

val_result = itm_eval(score_val_i2t, score_val_t2i, val_loader.dataset.txt2img, val_loader.dataset.img2txt)
print(val_result)
# test_result = itm_eval(score_test_i2t, score_test_t2i, test_loader.dataset.txt2img, test_loader.dataset.img2txt)
# print(test_result)

log_stats = {**{f'val_{k}': v for k, v in val_result.items()}}

print(log_stats)

Start evaluating
Computing features for evaluation...
Evaluation: [  0/501]  eta: 0:00:07    time: 0.0159  data: 0.0006  max mem: 6067
Evaluation: [ 50/501]  eta: 0:02:11    time: 0.2963  data: 0.0000  max mem: 6097
Evaluation: [100/501]  eta: 0:01:57    time: 0.2963  data: 0.0000  max mem: 6097
Evaluation: [150/501]  eta: 0:01:43    time: 0.2964  data: 0.0000  max mem: 6097
Evaluation: [200/501]  eta: 0:01:28    time: 0.2963  data: 0.0000  max mem: 6097
Evaluation: [250/501]  eta: 0:01:14    time: 0.2964  data: 0.0000  max mem: 6100
Evaluation: [300/501]  eta: 0:00:59    time: 0.2964  data: 0.0000  max mem: 6100
Evaluation: [350/501]  eta: 0:00:44    time: 0.2969  data: 0.0000  max mem: 6220
Evaluation: [400/501]  eta: 0:00:29    time: 0.2962  data: 0.0000  max mem: 6220
Evaluation: [450/501]  eta: 0:00:15    time: 0.2962  data: 0.0000  max mem: 6220
Evaluation: [500/501]  eta: 0:00:00    time: 0.2961  data: 0.0000  max mem: 6220
Evaluation: Total time: 0:02:28 (0.2958 s / it)
Evaluat